In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import newspaper
from newspaper import Article
from newspaper import Source
from newspaper import fulltext

In [2]:
test2021 = pd.read_csv('../data/test2_2021.csv')

In [3]:
# 16/03/2021 Error extracción text en Gamereactor

test2021[test2021['site'] == '3D Juegos'].head()

,site,url_link,author,game,company,genre,platform,text,score
1,3D Juegos,https://www.3djuegos.com/juegos/analisis/37840...,Sergio Martín,Tropico 6 - Análisis,None,None,PC PS4 XOne Switch Mac Linux,Gobierna una isla paradisíaca vayas donde vaya...,7.0
8,3D Juegos,https://www.3djuegos.com/juegos/analisis/37142...,Carlos Gallego,Maquette - Análisis,None,None,PC PS4 PS5,Maquette apela a los sentimientos de quien sos...,7.0
35,3D Juegos,https://www.3djuegos.com/juegos/analisis/36709...,Carlos Gallego,Call of the Sea - Análisis,None,None,PC XSeries XOne,Call of the Sea se aleja de la visión oscurant...,7.0
42,3D Juegos,https://www.3djuegos.com/juegos/analisis/37571...,Alberto Pastor,Scott Pilgrim contra el Mundo: El Videojuego -...,None,None,PC PS4 XOne Switch PS3 X360 Stadia,Diez años después de su lanzamiento original y...,7.0
43,3D Juegos,https://www.3djuegos.com/juegos/analisis/26242...,Álvaro C. / Toni P.,Cyberpunk 2077 - Análisis,None,None,PC PS4 PS5 XSeries XOne Stadia,"Esperado o no, Cyberpunk 2077 es un videojuego...",7.0


In [48]:
review_url = 'https://www.gamereactor.es/squad-analisis/'
#review_url = 'https://www.gamereactor.es/star-wars-squadrons-analisis'
review_html = requests.get(review_url).content
review_html[:50]

b' <!DOCTYPE html>\n<html lang="es">\n<head>\n\n\n<script'

In [49]:
soup = BeautifulSoup(review_html, 'lxml')

In [50]:
##Reparación 16/03/2021: se añae nuvo div, 'id: 'page0' en la extracción de article

article = soup.find('div', {'class': 'breadtext'}).find('div', {'id': 'page0'})
p_tags = article.find_all('p')
review = [tag.text for tag in p_tags]
review = ' '.join(review)

In [52]:
review[0:50]

'Anda que no ha llovido desde que jugamos Squad por'

In [53]:
# 16/03/2021 Error extracción todo en 3D Juegos

test2021[test2021['site'] == '3D Juegos'].head()

,site,url_link,author,game,company,genre,platform,text,score


In [58]:
review_url = 'https://www.3djuegos.com/juegos/analisis/37141/0/maquette/'
html = requests.get(review_url).content
html[:50]

b"<!DOCTYPE html>\n<html lang='es'>\n<head>\n<meta prop"

In [62]:
soup = BeautifulSoup(html, 'lxml')
p_tags = soup.find('div', {'class': 'lh27 url_lineas article_body0 mar_temp_0'}).find_all('p')


In [66]:
review = [tag.text for tag in p_tags]
review = ' '.join(review)
review[0:50]

'Maquette apela a los sentimientos de quien sostien'

In [65]:
author = soup.find('a', {'class': 'c7 n'}).text
author

'Carlos Gallego'

In [68]:
tdjuegos_platform(soup)

'PC PS4 PS5'

In [69]:
# 16/03/2021 Error score: Desde el 15 de diciembre de 2020, 3D Juegos no utiliza notas.

score = soup.find('div', {'class': 'nota_ana_3 fftext b nota_interior2'}).text
score = score.replace(',','.')
score = float(score)

AttributeError: 'NoneType' object has no attribute 'text'

In [70]:
# 16/03/2021 Error links: 3D Juegos ha cambiado la url de su landing de análisis. Reparción

''' Integrado en la función de links
    for i in range(pages):
        if i == 0:
            url = f"https://www.3djuegos.com/novedades/analisis/juegos/0f0f0f0/fecha/"
        else:
            url = f"https://www.3djuegos.com/novedades/analisis/juegos/{i}pf0f0f0/fecha/"'''

review_url = 'https://www.3djuegos.com/novedades/analisis/juegos/0f0f0f0/fecha/'
html = requests.get(review_url).content
html[:50]

b"<!DOCTYPE html>\n<html lang='es'>\n<head>\n<meta prop"

In [76]:
def tdjuegos_link_retrieve(num_pages):
    links = []
    titles = []
    
    # pages parser
    for i in range(num_pages):
        url = f"https://www.3djuegos.com/novedades/analisis/juegos/{i+100}pf0f0f0/fecha/"

        # building soup    
        html = requests.get(url).content
        soup = BeautifulSoup(html, 'lxml')
        articles = soup.find_all('h2')

        # links retrieve
        for a in articles:
            links.append(a.find('a')['href']) 
            titles.append(a.find('a')['title'])
    
    #Deleting non review links
#    for link in links:
#        if 'analisis' not in link:
#            links.remove(link)
                
    return links, titles

## Merge new and old scored texts

In [2]:
new_scored_texts = pd.read_csv('../new_scored_texts.csv')

In [3]:
scored_text_analy = pd.read_csv('../data/scored_texts_analytics.csv')

In [4]:
updated_scored_texts = pd.concat([new_scored_texts, scored_text_analy], ignore_index=True, sort=False)

In [5]:
len(updated_scored_texts)

16068

In [6]:
updated_scored_texts.head()

,site,author,game,score,score_adj,stars_mean,company,platform,genre,score_deviation
0,meristation,Rubén Vázquez,Peaky Blinders: Mastermind,5.7,2.85,3.00,Curve Digital,PC PS4 XBO NSW,"Aventura, Puzle",5.263158
1,meristation,Alejandro Castillo - @acastillo117,Tony Hawk's Pro Skater 1+2,8.5,4.25,3.80,Activision,PC XBO PS4 PS5 XBS NSW,Deportes,-10.588235
2,Vandal,Jorge Cano,Observer: System Redux - Análisis,7.8,3.90,3.50,Bloober Team,Xbox Series X/S PC PS5,Aventura narrativa Terror,-10.256410
3,meristation,Rocío Torrejón\nSeraPhiim_,Skul: The Hero Slayer,8.3,4.15,4.14,Neowiz,PC NSW,"Plataformas, Acción",-0.240964
4,Gamereactor,Alberto Garrido,Baldur's Gate III - Análisis Early Access,8.0,4.00,3.60,None,"PC, Stadia",RPG,-10.000000


In [9]:
len(updated_scored_texts.drop_duplicates())

16024

In [8]:
updated_scored_texts[updated_scored_texts.duplicated(keep=False)].sort_values(by=['game'])

,site,author,game,score,score_adj,stars_mean,company,platform,genre,score_deviation
63,revogamers,Marcos Catalán,Análisis de FEZ,9.0,4.50,3.67,Polytron,Nintendo Switch,"Aventura, Otros, Plataformas, Puzle",-18.444444
85,revogamers,Marcos Catalán,Análisis de FEZ,9.0,4.50,3.67,Polytron,Nintendo Switch,"Aventura, Otros, Plataformas, Puzle",-18.444444
139,revogamers,Marcos Catalán,Análisis de FEZ,9.0,4.50,3.67,Polytron,Nintendo Switch,"Aventura, Otros, Plataformas, Puzle",-18.444444
198,revogamers,Marcos Catalán,Análisis de FEZ,9.0,4.50,3.67,Polytron,Nintendo Switch,"Aventura, Otros, Plataformas, Puzle",-18.444444
383,revogamers,Marcos Catalán,Análisis de FEZ,9.0,4.50,3.67,Polytron,Nintendo Switch,"Aventura, Otros, Plataformas, Puzle",-18.444444
...,...,...,...,...,...,...,...,...,...,...
14620,meristation,Salva Fernàndez,Titanfall,9.3,4.65,3.81,None,PC 360 Xbox One,"Acción, First Person Shooter",-18.064516
7341,meristation,Salva Fernàndez,Ultimate Marvel vs. Capcom 3,7.0,3.50,3.33,None,PS3 360,"Acción, Arcade",-4.857143
13961,meristation,Salva Fernàndez,Ultimate Marvel vs. Capcom 3,7.0,3.50,3.33,None,PS3 360,"Acción, Arcade",-4.857143
11136,meristation,Will van Dijk,de Blob 2: The Underground,7.0,3.50,3.30,None,360 PS3 Wii,"Acción, Aventura",-5.714286


In [14]:
#updated_scored_texts.drop_duplicates().to_csv('../updated_scored_texts.csv', index=False)